### importing libraries:

In [1]:
import numpy as np
import tenseal as ts
import torch
from time import time
import pandas as pd
from sklearn.model_selection import train_test_split

### cleaning the dataset:

In [2]:
data=pd.read_csv("framingham.csv")
data.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
data.shape

(4238, 16)

##### removing rows with missing values:

In [4]:
data=data.dropna()
data.shape

(3656, 16)

##### removing unwanted columns:

In [5]:
data=data.drop(columns=["education","currentSmoker","BPMeds","diabetes","diaBP","BMI"])
data.head()

,male,age,cigsPerDay,prevalentStroke,prevalentHyp,totChol,sysBP,heartRate,glucose,TenYearCHD
0,1,39,0.0,0,0,195.0,106.0,80.0,77.0,0
1,0,46,0.0,0,0,250.0,121.0,95.0,76.0,0
2,1,48,20.0,0,0,245.0,127.5,75.0,70.0,0
3,0,61,30.0,0,1,225.0,150.0,65.0,103.0,1
4,0,46,23.0,0,0,285.0,130.0,85.0,85.0,0


##### balancing the data:

In [6]:
grouped=data.groupby('TenYearCHD')
grouped.head()

,male,age,cigsPerDay,prevalentStroke,prevalentHyp,totChol,sysBP,heartRate,glucose,TenYearCHD
0,1,39,0.0,0,0,195.0,106.0,80.0,77.0,0
1,0,46,0.0,0,0,250.0,121.0,95.0,76.0,0
2,1,48,20.0,0,0,245.0,127.5,75.0,70.0,0
3,0,61,30.0,0,1,225.0,150.0,65.0,103.0,1
4,0,46,23.0,0,0,285.0,130.0,85.0,85.0,0
5,0,43,0.0,0,1,228.0,180.0,77.0,99.0,0
6,0,63,0.0,0,0,205.0,138.0,60.0,85.0,1
15,0,38,20.0,0,1,221.0,140.0,95.0,70.0,1
17,0,46,20.0,0,0,291.0,112.0,80.0,89.0,1
25,1,47,20.0,0,0,294.0,102.0,62.0,66.0,1


In [7]:
data=grouped.apply(lambda x: x.sample(grouped.size().min(),random_state=73).reset_index(drop=True))
data.head()

male  age  cigsPerDay  prevalentStroke  prevalentHyp  totChol  \
TenYearCHD                                                                    
0          0     1   35        20.0                0             0    223.0   
           1     1   43         0.0                0             0    200.0   
           2     1   46         0.0                0             1    185.0   
           3     0   63         0.0                0             1    281.0   
           4     0   59         0.0                0             0    292.0   

              sysBP  heartRate  glucose  TenYearCHD  
TenYearCHD                                           
0          0  128.0       80.0     67.0           0  
           1  133.0       55.0     71.0           0  
           2  121.0       80.0     97.0           0  
           3  135.0       63.0     68.0           0  
           4  114.0       68.0     72.0           0

In [8]:
data.shape

(1114, 10)

In [9]:
data

male  age  cigsPerDay  prevalentStroke  prevalentHyp  totChol  \
TenYearCHD                                                                      
0          0       1   35        20.0                0             0    223.0   
           1       1   43         0.0                0             0    200.0   
           2       1   46         0.0                0             1    185.0   
           3       0   63         0.0                0             1    281.0   
           4       0   59         0.0                0             0    292.0   
...              ...  ...         ...              ...           ...      ...   
1          552     1   65         0.0                1             1    266.0   
           553     1   52         0.0                1             0    202.0   
           554     0   57         0.0                0             1    432.0   
           555     1   45        20.0                0             0    264.0   
           556     1   60        20.0                0             1    269.0   

                sysBP  heartRate  glucose  TenYearCHD  
TenYearCHD                                             
0          0    128.0       80.0     67.0           0  
           1    133.0       55.0     71.0           0  
           2    121.0       80.0     97.0           0  
           3    135.0       63.0     68.0           0  
           4    114.0       68.0     72.0           0  
...               ...        ...      ...         ...  
1          552  140.0       80.0     77.0           1  
           553  136.0       83.0     67.0           1  
           554  153.0       98.0     75.0           1  
           555  118.5       75.0     90.0           1  
           556  170.0       60.0     83.0           1  

[1114 rows x 10 columns]

In [10]:
y=torch.tensor(data["TenYearCHD"].values).float()
y

tensor([0., 0., 0.,  ..., 1., 1., 1.])

In [11]:
y=y.unsqueeze(1)

In [12]:
data=data.drop("TenYearCHD",axis=1)
data.head()

male  age  cigsPerDay  prevalentStroke  prevalentHyp  totChol  \
TenYearCHD                                                                    
0          0     1   35        20.0                0             0    223.0   
           1     1   43         0.0                0             0    200.0   
           2     1   46         0.0                0             1    185.0   
           3     0   63         0.0                0             1    281.0   
           4     0   59         0.0                0             0    292.0   

              sysBP  heartRate  glucose  
TenYearCHD                               
0          0  128.0       80.0     67.0  
           1  133.0       55.0     71.0  
           2  121.0       80.0     97.0  
           3  135.0       63.0     68.0  
           4  114.0       68.0     72.0

##### normalizing data:

In [13]:
data=(data-data.mean())/data.std()
x=torch.tensor(data.values).float()
x

tensor([[ 1.0436, -1.9451,  0.8790,  ..., -0.3666,  0.3485, -0.5414],
        [ 1.0436, -1.0146, -0.7572,  ..., -0.1627, -1.7064, -0.4173],
        [ 1.0436, -0.6656, -0.7572,  ..., -0.6521,  0.3485,  0.3899],
        ...,
        [-0.9574,  0.6138, -0.7572,  ...,  0.6531,  1.8280, -0.2931],
        [ 1.0436, -0.7819,  0.8790,  ..., -0.7541, -0.0625,  0.1726],
        [ 1.0436,  0.9628,  0.8790,  ...,  1.3465, -1.2955, -0.0447]])

In [14]:
x.shape

torch.Size([1114, 9])

In [15]:
y.shape

torch.Size([1114, 1])

##### dividing the dataset into training and testing:

In [16]:
x_train,x_test,y_train,y_test=train_test_split(x,y,stratify=y,test_size=0.3)

In [17]:
len(x_train),len(x_test),len(y_train),len(y_test)

(779, 335, 779, 335)

### Creating a neural network that facilitates Logisitic Regression Classification:

##### A class that creates a neural network for Logistic Regression:

In [18]:
class LR(torch.nn.Module):
    def __init__(self,n_features):
        super(LR,self).__init__()
        self.lr=torch.nn.Linear(n_features,1)
        
    def forward(self,x):
        out=torch.sigmoid(self.lr(x))
        return out

##### creating a neural network using LR class:

In [19]:
n_features=x_train.shape[1]
model=LR(n_features)

##### using gradient descent with a learning_rate=1

In [20]:
optim=torch.optim.SGD(model.parameters(),lr=1)

##### using Binary Cross Entropy Loss

In [21]:
criterion=torch.nn.BCELoss()

### Training the neural network on training data:

In [22]:
EPOCHS=5
def train(model,optim,criterion,x,y,epochs=EPOCHS):
    for e in range(1,epochs+1):
        optim.zero_grad()
        out=model(x)
        loss=criterion(out,y)
        loss.backward()
        optim.step()
        print(f"Loss at epoch {e}={loss.data}")
    return model

t1=time()
model=train(model,optim,criterion,x_train,y_train)
t2=time()
print(f'time taken to train the LR model on unencrypted data={t2-t1}ms')

Loss at epoch 1=0.6696200966835022
Loss at epoch 2=0.6299281120300293
Loss at epoch 3=0.6149693727493286
Loss at epoch 4=0.6080112457275391
Loss at epoch 5=0.6042595505714417
time taken to train the LR model on unencrypted data=0.0060079097747802734ms


### Calculating the accuracy of LR Neural Network:

In [23]:
def accuracy(model,x,y):
    out=model(x)
    correct=torch.abs(y-out)<0.5
    return correct.float().mean()

plain_accuracy=accuracy(model,x_test,y_test)
print(f"Accuracy on plain test_set:{plain_accuracy}")

Accuracy on plain test_set:0.6776119470596313
